In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import csv


from ipywidgets import interact_manual, interact, Checkbox, FloatSlider

from scipy.optimize import curve_fit

In [ ]:
cmap = [
    [0, 1.0000, 1.0000],
    [0, 0.8333, 1.0000],
    [0, 0.6667, 1.0000],
    [0, 0.5000, 1.0000],
    [0, 0.3333, 1.0000],
    [0, 0.1667, 1.0000],
    [0, 0, 1.0000],
    [0, 0, 0.8333],
    [0, 0, 0.6667]
]


In [ ]:
'''#count lines
nlines=0
with open('ratio_ratios.csv', 'r') as f:
    reader=csv.reader(f)
    for l in reader:
        nlines+=1
'''

with open('freqs.csv', 'r') as f:
    reader=csv.reader(f)
    for l in reader:
        freqs=np.array([float(freq) for freq in l])
        
n_levels=8
n_freqs=len(freqs)


data=np.zeros((n_freqs, n_levels))
with open('ratio_ratios.csv', 'r') as f:
    reader=csv.reader(f)
    j=0
    for l in reader:
        data_=np.array([float(x) for x in l])
        data[j]=data_
        j+=1

In [ ]:

@interact(a=(1, 5, 0.3), b=(8, 11, 0.1))
def myplot(a, b):
    for i in range(n_levels):
        pl.plot(freqs*1e-3, data[:, i], color=cmap[i])
    
    pl.plot(freqs*1e-3, 1+ 1/(1+np.exp(a*(freqs*1e-3-b) ) ), color='k')
    pl.xlabel('Frequency (kHz)')
    pl.xlim([5, 12])
    pl.ylim([0, 3])

### Sigmoid fits

In [ ]:
max_value=np.sqrt(2)

In [ ]:
data_m=np.maximum(data, 1)
data_m=np.minimum(data_m, max_value)
for i in range(n_levels):
    pl.plot(freqs*1e-3, data_m[:, i], color=cmap[i])
    pl.xlim([5, 12])
    pl.ylim([0, 3])

In [ ]:
def mysigm(x, a, b): 
    return 1+(max_value-1)*1/(1+np.exp(a*(x-b)))
    
def mysigm_J(x, a, b): #does not work
    sigm=mysigm(a, b, x)
    sigm_prime=-sigm*(1-sigm)
    return np.array([ (max_value-1)*sigm_prime*(x-b), -(max_value-1)*sigm_prime*a ]).T



In [ ]:
p=np.zeros((n_levels, 2))
for i in range(n_levels):
    
    y=data_m[:, i]
    ind_begin=np.sum(np.isnan(y[0:15]))
    p0=np.array([5-2.5*i/4, 9.2-i**2*0.2])
    popt, pcov = curve_fit(mysigm, freqs[ind_begin:17]*1e-3, y[ind_begin:17],p0, method='dogbox') #jac=mysigm_J)
    
    p[i]=popt

In [ ]:
p

In [ ]:
for i in range(n_levels):
    pl.plot(freqs*1e-3, data_m[:, i], color=cmap[i])
    
    pl.plot(freqs*1e-3, data[:, i], color=cmap[i], alpha=0.3)
    pl.plot(freqs*1e-3, 1+ (max_value-1)/(1+np.exp(p[i][0]*(freqs*1e-3-p[i][1]) ) ), color=cmap[i],  linestyle='--')
pl.xlabel('Frequency (kHz)')
pl.xlim([5, 12])
pl.ylim([0, 3])

In [ ]:

@interact(a=(1, 5, 0.3), b=(8, 11, 0.1))
def myplot(a, b):
    for i in range(n_levels):
        pl.plot(freqs*1e-3, data_m[:, i], color=cmap[i])
    
    pl.plot(freqs*1e-3, 1+ 0.5/(1+np.exp(a*(freqs*1e-3-b) ) ), color='k')
    pl.xlabel('Frequency (kHz)')
    pl.xlim([5, 12])
    pl.ylim([0, 3])

### TM_T/BM_T

In [ ]:
tm_bm_ratio =  3+np.arange(n_levels)/n_levels*(1.7- 3)

In [ ]:
for i in range(n_levels):
    pl.scatter(10+i*10, tm_bm_ratio[i], color=cmap[i], marker='+')
pl.ylabel('TM_T/BM_T')
pl.xlabel('Sound level')